In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 waste_cnt임)

In [119]:
##### 파일 불러오기 #####
df = pd.read_csv("C:\\Users\\asus\\BigContest\\files\\data_preprocessing\\1_food_waste_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'waste_cnt', 'waste_amt']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,waste_cnt,waste_amt
base_date,,,,
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850


In [120]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'waste_amt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,waste_cnt
base_date,,
2019-11-30,50110250,620
2019-12-31,50110250,8178
2020-01-31,50110250,13042
2020-02-29,50110250,12264
2020-03-31,50110250,14316
...,...,...
2021-02-28,50130620,4563
2021-03-31,50130620,4920
2021-04-30,50130620,5256


In [121]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110250  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,waste_cnt
base_date,
2019-11-30,620
2019-12-31,8178
2020-01-31,13042
2020-02-29,12264
2020-03-31,14316
2020-04-30,14513
2020-05-31,17614
2020-06-30,17365
2020-07-31,23417


In [122]:
# series 형으로 변환
sub_df.to_dict('series')

{'waste_cnt': base_date
 2019-11-30      620
 2019-12-31     8178
 2020-01-31    13042
 2020-02-29    12264
 2020-03-31    14316
 2020-04-30    14513
 2020-05-31    17614
 2020-06-30    17365
 2020-07-31    23417
 2020-08-31    24943
 2020-09-30    21807
 2020-10-31    21947
 2020-11-30    19990
 2020-12-31    19001
 2021-01-31    18369
 2021-02-28    17384
 2021-03-31    19901
 2021-04-30    20950
 2021-05-31    24046
 2021-06-30    25653
 Name: waste_cnt, dtype: int64}

# ARIMA

In [123]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [124]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -0.383704
p-value : 0.912784
Critical Values :
	1%: -4.137829
	5%: -3.154972
	10%: -2.714477


In [126]:
# 차분
diff_1 = sub_df_log.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 얼른 단톡에 보고하기 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -3.390853
p-value : 0.011275
Critical Values :
	1%: -4.137829
	5%: -3.154972
	10%: -2.714477


In [115]:
##### 이건 설치하는 거라 처음 돌릴 때 한 번만 주석 풀어서 설치하고 그 이후로는 필요없음! #####

#!pip install pmdarima

In [127]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

##### 여기서 아래쪽에 'Best model:  ARIMA(1,1,1)(0,0,0)[1] intercept' 이렇게 나오면 (1,1,1) 가져다 쓸거임 #####
##### ARIMA(1,1,1) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=37.161, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.11 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=nan, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=36.203, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=35.050, Time=0.13 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=36.979, Time=0.16 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=36.522, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.18 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=40.770, Time=0.21 sec
 ARIMA(3,1,0)(0,0,0)[1] intercept   : AIC=38.323, Time=0.06 sec
 AR

ARIMA(order=(1, 1, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [129]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.waste_cnt.values, order=(1,1,1))     ##### 여기 order=(1,1,1)에 위에서 나온 숫자 넣는거 #####
                                   ##### 그리고 sub_df_log.waste_cnt.values에서 가운데 waste_cnt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   19
Model:                 ARIMA(1, 1, 1)   Log Likelihood                 -13.524
Method:                       css-mle   S.D. of innovations              0.452
Date:                Fri, 10 Sep 2021   AIC                             35.049
Time:                        00:54:13   BIC                             38.827
Sample:                             1   HQIC                            35.688
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2807      0.244      1.152      0.249      -0.197       0.758
ar.L1.D.y      0.1506      0.374      0.403      0.687      -0.582       0.883
ma.L1.D.y      0.9998      0.714      1.400      0.1

C:\Users\asus\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [130]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값에 로그씌운거임 #####
##### 7월 : 10.05681231, 8월 : 10.28080669 #####
##### 이거 지수함수에 넣어서 원래값으로 돌려야 함 -> 각각 가져다 밑에 넣기 #####

(array([10.05681231, 10.28080669]), array([0.45225605, 1.07253507]), array([[ 9.17040674, 10.94321787],
       [ 8.17867659, 12.3829368 ]]))


In [137]:
# 이거 위에서 로그씌울 때 np.log1p 이거 써서 지수함수에 넣은 다음에 1 뺀거임
# https://rfriend.tistory.com/295
# 궁금하면 이 링크 들어가서 읽어보세요 질문받기 싫어서 미리 써놓음
# 밑에 어떻게 되는지 예제코드도 넣었으니까 제발 질문 안해주길 ㅎ

print( np.exp(10.05681231) - 1 )   ##### 괄호 안에 7월 숫자 넣기 #####
print( np.exp(10.28080669) - 1 )   ##### 괄호 안에 8월 숫자 넣기 #####

##### 이 값 가져다 엑셀파일에 입력하기 #####

23313.069767720848
29166.393110116605


In [138]:
a = 134
lna = np.log1p(a)  # a에 자연로그씌운거
ea = np.exp(lna)  # 다시 지수함수에 넣은거
print(a, lna, ea)  # 결과가 1 크게 나옴

134 4.90527477843843 135.00000000000003
